# Day 5: Supply Stacks
https://adventofcode.com/2022/day/5

In [2]:
let lines: Vec<String> = std::fs::read_to_string("input/day05.txt")
    .expect("Could not read file")
    .lines()
    .map(|line| line.to_owned())
    .collect();

In [3]:
let example: Vec<String> = "    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2".lines().map(|line| line.to_owned()).collect();

In [4]:
fn parse_stacks(input: &[String]) -> Vec<Vec<char>> {
    // Each stack has 3 chars, and there is a single space in between
    // Thus, there are 4*n - 1 chars for n columns
    let count = (input[0].len() + 1) / 4;
    assert!(input.iter().all(|line| line.len() == (4 * count - 1)));
    
    // Verify the structure of the bottom row with the stack numbers
    let last_row = input.last().unwrap();
    for (i, c) in last_row.chars().enumerate() {
        if i % 4 == 1 {
            let stack_id = (i - 1) / 4 + 1;
            assert_eq!(c, char::from_digit(stack_id.try_into().unwrap(), 10).unwrap());
        } else {
            assert_eq!(c, ' ');
        }
    }
    
    // Parse the stacks. TODO: this could also be done in a more functionaly style, without mutability.
    let mut stacks: Vec<Vec<char>> = Vec::new();
    stacks.resize(count, Vec::new());
    
    for line in input[..input.len() - 1].iter().rev() {
        for i in 0..count {
            let c = line.chars().nth(4 * i + 1).unwrap();
            if c != ' ' {
                stacks[i].push(c);
            }
        }
    }
    
    stacks
}

In [5]:
#[derive(Debug)]
struct Move {
    amount: usize,
    from: usize,
    to: usize
}

In [6]:
:dep regex = "1.7.0"

In [7]:
use regex::Regex;

In [8]:
fn parse_moves(input: &[String]) -> Vec<Move> {
    let re = Regex::new("move (\\d+) from (\\d+) to (\\d+)").unwrap();
    input.iter()
        .map(|line| {
            let captures = re.captures(line).unwrap();
            let get_item = |i| captures.get(i).unwrap().as_str().parse::<usize>().unwrap();
            Move {
                amount: get_item(1),
                from: get_item(2),
                to: get_item(3)
            }
        })
        .collect()            
}

In [9]:
fn parse(input: &[String]) -> (Vec<Vec<char>>, Vec<Move>) {
    let mut s = input.split(|line| line.len() == 0);
    let stacks = s.next().unwrap().to_vec();
    let moves = s.next().unwrap().to_vec();
    assert!(s.next() == None);
    (parse_stacks(&stacks), parse_moves(&moves))
}

In [10]:
impl Move {
    // Crates are moved one by one
    fn apply_part1(self: &Self, stacks: &mut Vec<Vec<char>>) {
        for _ in 0..self.amount {
            // Note that indices in Move are 1-based
            let item = stacks[self.from - 1].pop().unwrap();
            stacks[self.to - 1].push(item);
        }
    }
}

In [11]:
fn part1(input: &[String]) -> String {
    let (mut stacks, moves) = parse(input);
    for m in moves {
        m.apply_part1(&mut stacks);
    }
    
    stacks.iter()
        .map(|stack| stack.last().unwrap())
        .collect()
}

In [12]:
part1(&example)

"CMZ"

In [13]:
part1(&lines)

"CFFHVVHNC"

## Part 2

In [14]:
impl Move {
    // Moving multiple crates maintains their order now.
    fn apply_part2(self: &Self, stacks: &mut Vec<Vec<char>>) {
        // Note that indices in Move are 1-based
        let from = &stacks[self.from - 1];
        
        // TODO: if we do not make a Vec out of the slice, the borrow checker complains
        //let items = &from[from.len() - self.amount..];
        let items = &from[from.len() - self.amount..].to_vec();
        stacks[self.to - 1].extend_from_slice(&items);
        let mut from = &mut stacks[self.from - 1];
        from.truncate(from.len() - self.amount);
    }
}

In [15]:
fn part2(input: &[String]) -> String {
    let (mut stacks, moves) = parse(input);
    for m in moves {
        m.apply_part2(&mut stacks);
    }
    
    stacks.iter()
        .map(|stack| stack.last().unwrap())
        .collect()
}

In [16]:
part2(&example)

"MCD"

In [17]:
part2(&lines)

"FSZWBPTBG"